# UBER- Eats Order Tracking Partner Performance Evaluation

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_orders = pd.read_csv('Data/016/fct_orders.csv')
df_orders.head()

,order_id,delivery_partner_id,delivery_partner_name,expected_delivery_time,actual_delivery_time,order_date
0,1,1,Alice,2024-01-05 12:00:00.000000,2024-01-05 11:45:00.000000,2024-01-05
1,2,1,Alice,2024-01-12 13:00:00.000000,2024-01-12 13:00:00.000000,2024-01-12
2,3,1,Alice,2024-01-20 12:30:00.000000,2024-01-20 12:35:00.000000,2024-01-20
3,4,2,Bob,2024-01-07 14:00:00.000000,2024-01-07 13:50:00.000000,2024-01-07
4,5,2,Bob,2024-01-15 15:00:00.000000,2024-01-15 15:05:00.000000,2024-01-15


# Pregunta 1

### ¿Cuál es el porcentaje de pedidos entregados a tiempo en enero de 2024? Considera que un pedido está a tiempo si su actual_delivery_time es menor o igual a su expected_delivery_time. Esto nos ayudará a evaluar la precisión general del seguimiento.

In [3]:
df_enero = df_orders[
    (df_orders['order_date'].between('2024-01-01','2024-01-31'))
]

precision = ((df_enero['actual_delivery_time'] <= df_enero['expected_delivery_time']).mean() * 100).round(2)

precision



np.float64(66.67)

```SQL
SELECT
    ROUND(
        AVG(CASE WHEN actual_delivery_time <= expected_delivery_time THEN 1.0 ELSE 0.0 END) * 100,
        2
    ) AS percentage_delivered_ontime
FROM fct_orders
WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31';
```

# Pregunta 2

### Haz una lista de los 5 mejores repartidores en enero de 2024, clasificados por el mayor porcentaje de entregas a tiempo. Utiliza el campo delivery_partner_name de los registros. Esto nos ayudará a identificar qué socios tienen el mejor desempeño.

In [11]:
ranking_repartidores = (
    df_enero.groupby('delivery_partner_name')['actual_delivery_time']
    .apply(lambda x: (x <= df_enero.loc[x.index, 'expected_delivery_time']).mean() * 100)
    .round(2)
    .reset_index(name='percentage_on_time')
    .sort_values(by='percentage_on_time', ascending=False)
    .head(5)
)

ranking_repartidores

,delivery_partner_name,percentage_on_time
5,Frank,100.00
0,Alice,66.67
1,Bob,66.67
2,Charlie,66.67
4,Eve,66.67


```SQL
SELECT
    delivery_partner_name,
    ROUND(AVG(CASE WHEN actual_delivery_time <= expected_delivery_time THEN 1.0 ELSE 0.0 END)*100,2) AS percentage_time_deliveries
FROM fct_orders
WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31'
GROUP BY delivery_partner_name
ORDER BY percentage_time_deliveries desc
LIMIT 5;
```

# Pregunta 3

### Identifica al repartidor (o repartidores) en enero de 2024 cuyo porcentaje de entregas a tiempo sea inferior al 50%. Devuelve sus nombres de repartidor en mayúsculas (UPPERCASE). Necesitamos trabajar con estos socios repartidores para mejorar sus tasas de entrega a tiempo.

In [14]:
resultado3 = (df_enero.groupby('delivery_partner_name')['actual_delivery_time']
.apply(lambda x: (x <= df_enero.loc[x.index, 'expected_delivery_time']).mean() * 100)
[lambda x: x < 50]
.reset_index())

resultado3

,delivery_partner_name,actual_delivery_time
0,Dawn,33.333333


```SQL
SELECT
    UPPER (delivery_partner_name) AS partner_name_upper,
    ROUND(
    AVG(CASE WHEN actual_delivery_time <= expected_delivery_time THEN 1.0 ELSE 0.0 END) * 100, 2
    ) AS percentage_on_time
FROM fct_orders
WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31'
GROUP BY delivery_partner_name
HAVING AVG(CASE WHEN actual_delivery_time <= expected_delivery_time THEN 1.0 ELSE 0.0 END) * 100 < 50;
```